# CSF ANALYSIS FOR PPMI data


Created: 2018-08-07    
By: Hirotaka Iwaki

## GLMM analysis

### Analysis models

In [55]:
%%bash
rm -f models.txt
echo 'MODEL;OUTCOME;COVS;GENOTYPEFILE;PHENOTYPEFILE;OUTFILE' > models.txt
MODEL=lnmxi
DATASET=PPMI
for GENOTYPEFILE in $(ls ../dataset/$DATASET/maf01rsq3_20Kcut/*);do
    for PHENOTYPEFILE in $(ls dataset/$DATASET/pheno/*);do
        for OUTCOME in logASYN ivnASYN rawASYN;do
            if [[ $PHENOTYPEFILE == *"all.txt" ]];then
                COVS="HEMO,FEMALE,AGE,CASE,PC1,PC2"
            elif [[ $PHENOTYPEFILE == *"control.txt" ]];then
                COVS="HEMO,FEMALE,AGE,PC1,PC2"
            else 
                COVS="HEMO,FEMALE,AGE,AAO,PC1,PC2"
            fi
            OUTFOLDER=dataset/$DATASET/GWAS/$MODEL/$OUTCOME
            echo "\
$MODEL:$OUTCOME:$COVS:$GENOTYPEFILE:$PHENOTYPEFILE:$OUTFOLDER" >> models.txt
        done
    done
done

### Rcode for analysis

In [56]:
%%bash
echo '
txt <- commandArgs(trailingOnly = TRUE)
library(data.table);library(dplyr)
ANALYSIS=strsplit(txt, ":")[[1]][1]
OUTCOME=strsplit(txt, ":")[[1]][2]
COVPC=strsplit(txt, ":")[[1]][3] %>% gsub(",", "+", .) 
INFILE=paste("gzcat -f", strsplit(txt, ":")[[1]][4])
DATASET=strsplit(txt, ":")[[1]][5]
OUTFOLDER=strsplit(txt, ":")[[1]][6]
SNPset = fread(cmd=INFILE)
SNPs = names(SNPset)[-c(1:2)] 
# Read phenotype
library(lme4)
cohort=fread(DATASET) %>% select(c("IID", OUTCOME, strsplit(COVPC, "\\+")[[1]])) %>% filter(complete.cases(.))
# rescaling
cohort[,(strsplit(COVPC, "\\+")[[1]])] = as.data.frame(scale(cohort[,(strsplit(COVPC, "\\+")[[1]])]))
# Merge data
cohort_snp = left_join(cohort, SNPset, by = "IID")
# ANALYSIS
test.listfunc = function(x){
  MODEL = paste(OUTCOME, "~", "`", SNPs[x], "`+", paste(COVPC, collapse="+"), "+(1|IID)", sep = "")
  testLmer = try(lmer(eval(parse(text = MODEL)), data = cohort_snp),silent = T)
  if(class(testLmer)[1]=="try-error"){
    sumstat=c(SNPs[x], "NoConverge", rep(NA,4))
  }else{
    temp = summary(testLmer)
    temp1 = temp$coefficients
    if(grep(substr(SNPs[x],1,7), rownames(temp1)) %>% length == 0){
      sumstat=c(SNPs[x], "Dropped", rep(NA,4))
    }else{
      RES = temp1[2,] # The first row is intercept
      PV_APPROX = 2 * pnorm(-abs(RES[3])) # df is large enough for approximation
      OBS_N = paste(length(temp$residuals), "_", temp$ngrps, sep="")
      sumstat <- c(SNPs[x], OBS_N, RES[3], RES[1], RES[2], PV_APPROX)
    }
  }
  return(sumstat)
}
temp = lapply(1:length(SNPs), test.listfunc)
temp2 = do.call(rbind, temp) %>% data.frame%>%filter(complete.cases(.))
names(temp2)=c("POS_A2_A1", "EVENT_OBS", "Tvalue", "BETA", "SE", "P")
dir.create(OUTFOLDER, recursive = T, showWarnings = F)
FILENAME = strsplit(INFILE, "/")[[1]][5] %>% substr(.,1,nchar(.)-3) %>% sub("\\.", "_", .)
write.table(temp2, paste(OUTFOLDER, FILENAME, sep="/"), row.names = F, quote = F, sep = "\t")
line=paste("Rscript --vanilla t/", ANALYSIS, ".R ", txt, sep="")
write(line,file=paste("finish_", ANALYSIS, ".txt", sep=""),append=TRUE)
' > t/lnmxi.R

# Analysis
### Only among cases

In [1]:
%%bash
head -n2 models.txt
grep 'logASYN' models.txt | grep case.txt | sed 's/^/Rscript --vanilla t\/lnmxi.R /' > t/swarm_logASYN.txt
swarm -f t/swarm_logASYN.txt --time=2:00:00 -g 3 -p 2 -b 2 --logdir t/swarm --module R --devel

MODEL;OUTCOME;COVS;GENOTYPEFILE;PHENOTYPEFILE;OUTFILE
lnmxi:logASYN:HEMO,FEMALE,AGE,CASE,PC1,PC2:../dataset/PPMI/maf01rsq3_20Kcut/cut1.0.txt.gz:dataset/PPMI/pheno/all.txt:dataset/PPMI/GWAS/lnmxi/logASYN
Loading modules R
150 commands run in 38 subjobs, each command requiring 3 gb and 1 thread, packing 2 processes per subjob, running 2 processes serially per subjob
sbatch --array=0-37 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=6144 --partition=norm --time=04:00:00 /spin1/swarm/iwakih2/h7E2uRN15Q/swarm.batch


In [58]:
%%bash
swarm -f t/swarm_logASYN.txt --time=2:00:00 -g 3 -p 2 -b 2 --logdir t/swarm --module R --devel

10803817


### Summarize analysis results
Filter out |BETA|<5, MAF>0.05

In [75]:
%%bash
rm -f t/populate*.txt
DATASET="PPMI"
RESULT="GWAS"
MODEL="lnmxi"
OUTCOME="logASYN"
B_THRES=5
echo "
cat dataset/$DATASET/$RESULT/$MODEL/$OUTCOME/*.txt | sed 's/_/\t/g' | cut -f 1-3,7-9 |\
 awk 'BEGIN{OFS=\"\t\"}{print \$1,\$3,\$2,\$4,\$5,\$6}' | awk 'NR==1{print;next} -$B_THRES < \$4 && \$4 < $B_THRES{print|\"LANG=C sort\"}' |\
 LANG=C join -t$'\t' --header - ../dataset/$DATASET/maf01rsq3_sorted.txt|\
 awk 'NR==1{print;next}\$2==\$8{print}' | cut -f 1-6,9-10 | awk 'NR==1{print;next}\$7>0.05{print}'>\
 dataset/$DATASET/$RESULT/$MODEL/$OUTCOME/filt.tab" \
 > t/populate_"$RESULT".txt

head -n2 t/populate_"$RESULT".txt
swarm -f t/populate_"$RESULT".txt --time=0:01:00 -g 2 --logdir swarm/t --partition=quick --devel


cat dataset/PPMI/GWAS/lnmxi/logASYN/*.txt | sed 's/_/\t/g' | cut -f 1-3,7-9 | awk 'BEGIN{OFS="\t"}{print $1,$3,$2,$4,$5,$6}' | awk 'NR==1{print;next} -5 < $4 && $4 < 5{print|"LANG=C sort"}' | LANG=C join -t$'\t' --header - ../dataset/PPMI/maf01rsq3_sorted.txt| awk 'NR==1{print;next}$2==$8{print}' | cut -f 1-6,9-10 | awk 'NR==1{print;next}$7>0.05{print}'> dataset/PPMI/GWAS/lnmxi/logASYN/filt.tab
1 commands run in 1 subjob, each command requiring 2 gb and 1 thread, allocating 1 core and 2 cpus
sbatch --array=0-0 --job-name=swarm --output=swarm/t/swarm_%A_%a.o --error=swarm/t/swarm_%A_%a.e --cpus-per-task=1 --mem=2048 --partition=quick --time=01:00 /spin1/swarm/iwakih2/E0NApo49op/swarm.batch


In [76]:
%%bash
cat dataset/PPMI/GWAS/lnmxi/logASYN/*.txt | sed 's/_/\t/g' | cut -f 1-3,7-9 | awk 'BEGIN{OFS="\t"}{print $1,$3,$2,$4,$5,$6}' |\
awk 'NR==1{print;next} -5 < $4 && $4 < 5{print|"LANG=C sort"}' | LANG=C join -t$'\t' --header - ../dataset/PPMI/maf01rsq3_sorted.txt|\
awk 'NR==1{print;next}$2==$8{print}' | cut -f 1-6,9-10 | awk 'NR==1{print;next}$7>0.05{print}'> dataset/PPMI/GWAS/lnmxi/logASYN/filt.tab

In [50]:
%%bash
awk '$6<5e-7' dataset/PPMI/GWAS/lnmxi/logASYN_case/filt.tab

4:87776334	G	T	0.268169660596023	0.0531021224494774	4.41638395965854e-07	0.07409	0.95845
4:87779200	C	T	0.267816901603139	0.0530257588149491	4.40206917632107e-07	0.07412	0.96079
4:87779724	T	C	-0.258854902073759	0.0497119591444116	1.91819731415195e-07	0.91924	0.94901
4:87780869	C	T	0.267153119859032	0.0529940221523419	4.62637380230938e-07	0.07438	0.95914
4:87782506	A	C	0.267193109552545	0.0529286035153237	4.46039163988488e-07	0.07410	0.96459
4:87790222	T	A	0.265861905852615	0.0528041269087502	4.78173636816479e-07	0.07438	0.96603
4:87790756	A	G	0.265768736667505	0.0528003467871526	4.81690932222062e-07	0.07446	0.96544
4:87790805	T	C	0.266052550099932	0.0527787863350308	4.63349111832365e-07	0.07413	0.96976
4:87795055	C	A	0.265040379631759	0.0526226558031225	4.73821335018687e-07	0.07439	0.97214
8:124689840	C	G	-0.275543703905232	0.0530412229279553	2.04832924136123e-07	0.19356	0.45366


In [1]:
%%bash
awk '$6<5e-7{print $1}' dataset/PPMI/GWAS/lnmxi/logASYN_case/filt.tab > dataset/PPMI/GWAS/lnmxi/case_subhits.txt

## Unified model with cases and controls

In [80]:
%%bash
head -n2 models.txt
grep 'logASYN' models.txt | grep all.txt | sed 's/^/Rscript --vanilla t\/lnmxi.R /' > t/swarm_logASYN.txt
swarm -f t/swarm_logASYN.txt --time=2:00:00 -g 3 -p 2 -b 2 --logdir t/swarm --module R --devel

MODEL;OUTCOME;COVS;GENOTYPEFILE;PHENOTYPEFILE;OUTFILE
lnmxi:logASYN:HEMO,FEMALE,AGE,CASE,PC1,PC2:../dataset/PPMI/maf01rsq3_20Kcut/cut1.0.txt.gz:dataset/PPMI/pheno/all.txt:dataset/PPMI/GWAS/lnmxi/logASYN
Loading modules R
150 commands run in 38 subjobs, each command requiring 3 gb and 1 thread, packing 2 processes per subjob, running 2 processes serially per subjob
sbatch --array=0-37 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=6144 --partition=norm --time=04:00:00 /spin1/swarm/iwakih2/UmPn2OY4PO/swarm.batch


In [81]:
%%bash
swarm -f t/swarm_logASYN.txt --time=2:00:00 -g 3 -p 2 -b 2 --logdir t/swarm --module R --devel

10833130


In [1]:
%%bash
cat dataset/PPMI/GWAS/lnmxi/logASYN/*.txt | sed 's/_/\t/g' | cut -f 1-3,7-9 | awk 'BEGIN{OFS="\t"}{print $1,$3,$2,$4,$5,$6}' | awk 'NR==1{print;next} -5 < $4 && $4 < 5{print|"LANG=C sort"}' | LANG=C join -t$'\t' --header - ../dataset/PPMI/maf01rsq3_sorted.txt| awk 'NR==1{print;next}$2==$8{print}' | cut -f 1-6,9-10 | awk 'NR==1{print;next}$7>0.05{print}'> dataset/PPMI/GWAS/lnmxi/logASYN/filt.tab

In [2]:
%%bash
awk '$6<5e-7' dataset/PPMI/GWAS/lnmxi/logASYN/filt.tab | head

4:87829251	G	A	0.216182405089796	0.0428984034458618	4.66982704810816e-07	0.07835	0.97379
4:87874574	C	A	0.219050161093464	0.0435291985907932	4.84736115349288e-07	0.08004	0.92543
4:87905696	T	G	0.220646591993408	0.0438924412602532	4.98254377104771e-07	0.08046	0.90305
8:124688861	A	G	-0.229111866068758	0.0454550174851971	4.64538681399518e-07	0.14954	0.49466


In [3]:
%%bash
mv dataset/PPMI/GWAS/lnmxi/logASYN dataset/PPMI/GWAS/lnmxi/logASYN_all

### Analysis only with controls

In [4]:
%%bash
head -n2 models.txt
grep 'logASYN' models.txt | grep control.txt | sed 's/^/Rscript --vanilla t\/lnmxi.R /' > t/swarm_logASYN.txt
swarm -f t/swarm_logASYN.txt --time=2:00:00 -g 3 -p 2 -b 2 --logdir t/swarm --module R --devel
swarm -f t/swarm_logASYN.txt --time=2:00:00 -g 3 -p 2 -b 2 --logdir t/swarm --module R

MODEL;OUTCOME;COVS;GENOTYPEFILE;PHENOTYPEFILE;OUTFILE
lnmxi:logASYN:HEMO,FEMALE,AGE,CASE,PC1,PC2:../dataset/PPMI/maf01rsq3_20Kcut/cut1.0.txt.gz:dataset/PPMI/pheno/all.txt:dataset/PPMI/GWAS/lnmxi/logASYN
Loading modules R
150 commands run in 38 subjobs, each command requiring 3 gb and 1 thread, packing 2 processes per subjob, running 2 processes serially per subjob
sbatch --array=0-37 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=6144 --partition=norm --time=04:00:00 /spin1/swarm/iwakih2/uKGxVPYPvz/swarm.batch
10919729


In [2]:
%%bash
cat dataset/PPMI/GWAS/lnmxi/logASYN/*.txt | sed 's/_/\t/g' | cut -f 1-3,7-9 | awk 'BEGIN{OFS="\t"}{print $1,$3,$2,$4,$5,$6}' | awk 'NR==1{print;next} -5 < $4 && $4 < 5{print|"LANG=C sort"}' | LANG=C join -t$'\t' --header - ../dataset/PPMI/maf01rsq3_sorted.txt| awk 'NR==1{print;next}$2==$8{print}' | cut -f 1-6,9-10 | awk 'NR==1{print;next}$7>0.05{print}'> dataset/PPMI/GWAS/lnmxi/logASYN/filt.tab
mv dataset/PPMI/GWAS/lnmxi/logASYN dataset/PPMI/GWAS/lnmxi/logASYN_control
awk '$6<5e-7' dataset/PPMI/GWAS/lnmxi/logASYN_control/filt.tab | head

15:77854352	T	G	0.273710327142572	0.0536199021587898	3.31423863072548e-07	0.32237	0.62113
15:77854678	T	C	0.273817098510611	0.0536459083987464	3.3227366123378e-07	0.32236	0.62054
15:77854876	C	A	0.273709129951563	0.0536483522218505	3.36240490728656e-07	0.32232	0.62037
15:77856571	A	T	0.273513279460726	0.0537695172057713	3.64209065123717e-07	0.32290	0.61679


In [5]:
%%bash
head -n1 dataset/PPMI/GWAS/lnmxi/logASYN_control/filt.tab
grep '4:87779724' dataset/PPMI/GWAS/lnmxi/logASYN_control/filt.tab
grep '4:87874574' dataset/PPMI/GWAS/lnmxi/logASYN_control/filt.tab

POS	A1	A2	BETA	SE	P	ALT_Frq	Rsq
4:87779724	T	C	-0.113251894793522	0.077916474109751	0.146083912599818	0.91924	0.94901
4:87874574	C	A	0.162000912608873	0.0778229466215886	0.037373539607659	0.08004	0.92543


In [2]:
%%bash
head -n1 dataset/PPMI/GWAS/lnmxi/logASYN_control/filt.tab
grep -f dataset/PPMI/GWAS/lnmxi/case_subhits.txt dataset/PPMI/GWAS/lnmxi/logASYN_control/filt.tab

POS	A1	A2	BETA	SE	P	ALT_Frq	Rsq
4:87776334	G	T	0.113062613427803	0.0806329054713359	0.160858666205001	0.07409	0.95845
4:87779200	C	T	0.113890432349514	0.0805503104304594	0.157389995711666	0.07412	0.96079
4:87779724	T	C	-0.113251894793522	0.077916474109751	0.146083912599818	0.91924	0.94901
4:87780869	C	T	0.114190147917796	0.0805019984493301	0.156051858296662	0.07438	0.95914
4:87782506	A	C	0.114738787148842	0.0804220192073666	0.153663903161291	0.07410	0.96459
4:87790222	T	A	0.115839049791902	0.0802668574384782	0.148971433410482	0.07438	0.96603
4:87790756	A	G	0.116086214283221	0.0802601598620488	0.148072309952289	0.07446	0.96544
4:87790805	T	C	0.11599059904781	0.0802460453271181	0.148335190788936	0.07413	0.96976
4:87795055	C	A	0.117516748765643	0.0800746928673024	0.142215862676004	0.07439	0.97214
8:124689840	C	G	-0.0904154328480027	0.0761747144689051	0.235248091579088	0.19356	0.45366


## Meta analysis with cases and controls

In [44]:
%%bash
rm -f models_meta.txt
MODEL="lnmxi"
OUTCOME="logASYN"
OUTDIR="meta/$MODEL/$OUTCOME"
mkdir -p $OUTDIR
echo "$MODEL:$OUTCOME:$OUTDIR:" > models_meta_line.txt
for i in $(ls dataset/PPMI/GWAS/$MODEL/logASYN_*/filt.tab);do
    sed -i "s|\$|$i,|" models_meta_line.txt
done
cat models_meta_line.txt >> models_meta.txt
rm  models_meta_line.txt

# Create Metal file
for i in $(cat models_meta.txt); do
    MODEL=$(echo $i | cut -d':' -f1)
    OUTCOME=$(echo $i | cut -d':' -f2)
    OUTDIR=$(echo $i | cut -d':' -f3)
    INFILEs=$(echo $i | cut -d':' -f4 | sed 's|,|\t|g')
    rm -f $OUTDIR/metal.txt
    for INFILE in $INFILEs;do
        echo "\
MARKER POS
ALLELE A1 A2
EFFECT BETA
STDERR SE
PVALUE P
PROCESS $INFILE" >> $OUTDIR/metal.txt
    done
    sed -i "1 iSCHEME STDERR" meta/$MODEL/$OUTCOME/metal.txt
    sed -i "\$ aOUTFILE $OUTDIR/res .tbl\nANALYZE HETEROGENEITY\nQUIT" meta/$MODEL/$OUTCOME/metal.txt
done

In [45]:
%%bash
module load metal
metal meta/lnmxi/logASYN/metal.txt

MetaAnalysis Helper - (c) 2007 - 2009 Goncalo Abecasis

# This program faciliates meta-analysis of genome-wide association studies.
# Commonly used commands are listed below:
#
# Options for describing input files ...
#   SEPARATOR        [WHITESPACE|COMMA|BOTH|TAB] (default = WHITESPACE)
#   COLUMNCOUNTING   [STRICT|LENIENT]            (default = 'STRICT')
#   MARKERLABEL      [LABEL]                     (default = 'MARKER')
#   ALLELELABELS     [LABEL1 LABEL2]             (default = 'ALLELE1','ALLELE2')
#   EFFECTLABEL      [LABEL|log(LABEL)]          (default = 'EFFECT')
#   FLIP
#
# Options for filtering input files ...
#   ADDFILTER        [LABEL CONDITION VALUE]     (example = ADDFILTER N > 10)
#                    (available conditions are <, >, <=, >=, =, !=, IN)
#   REMOVEFILTERS
#
# Options for sample size weighted meta-analysis ...
#   WEIGHTLABEL      [LABEL]                     (default = 'N')
#   PVALUELABEL      [LABEL]                     (default = 'PVALUE')
#   DEFAUL

[+] Loading metal  2017-12-21 


In [52]:
%%bash
head -n1 meta/lnmxi/logASYN/res1.tbl
awk '$6<5e-7' meta/lnmxi/logASYN/res1.tbl

MarkerName	Allele1	Allele2	Effect	StdErr	P-value	Direction	HetISq	HetChiSq	HetDf	HetPVal
4:87850830	a	c	-0.2245	0.0432	2.095e-07	--	8.2	1.089	1	0.2967
4:87938820	t	c	0.2255	0.0446	4.149e-07	++	7.7	1.083	1	0.2979
4:87821274	t	c	0.2211	0.0429	2.619e-07	++	6.2	1.066	1	0.3018
4:87863394	c	g	-0.2252	0.0434	2.129e-07	--	10.7	1.120	1	0.29
4:87804090	c	g	-0.2209	0.0430	2.814e-07	--	8.0	1.087	1	0.297
4:87810531	t	c	-0.2205	0.0429	2.803e-07	--	7.0	1.076	1	0.2997
4:87874574	a	c	-0.2274	0.0436	1.844e-07	--	2.7	1.028	1	0.3106
4:87779724	t	c	-0.2167	0.0419	2.322e-07	--	59.7	2.482	1	0.1152
4:87800067	t	c	0.2215	0.0431	2.808e-07	++	9.2	1.102	1	0.2939
4:87863353	t	c	-0.2252	0.0434	2.117e-07	--	10.5	1.117	1	0.2905
4:87822078	a	g	-0.2212	0.0429	2.59e-07	--	6.2	1.066	1	0.3018
4:87817916	a	g	0.2209	0.0429	2.662e-07	++	6.5	1.069	1	0.3011
4:87838529	a	g	0.2237	0.0431	2.079e-07	++	6.6	1.070	1	0.3009
4:87816557	a	g	-0.2209	0.0429	2.662e-07	--	6.3	1.067	1	0.3016
4:87929485	a	g	-0.2251	0.0439	2.958e-07	--	0.0	0.

In [2]:
%%bash
zless meta/lnmxi/logASYN/res1.tbl.gz | awk 'NR==1{print;next}NR>1{print|"LANG=C sort"}' |\
 LANG=C join -t$'\t' --header - ../dataset/PPMI/maf01rsq3_sorted.txt | cut -f1-11,14-15 |\
 LANG=C join -t$'\t' --header - ../tool/HRC_RS_conversion_sorted.txt | cut -f-14 > meta/lnmxi/logASYN/wID.txt
head meta/lnmxi/logASYN/wID.txt

MarkerName	Allele1	Allele2	Effect	StdErr	P-value	Direction	HetISq	HetChiSq	HetDf	HetPVal	ALT_Frq	Rsq	ID
10:100000625	a	g	-0.0063	0.0253	0.8036	-+	81.1	5.288	1	0.02147	0.43103	0.82564	rs7899632
10:100000645	a	c	-0.0426	0.0307	0.1656	--	0.0	0.508	1	0.476	0.19392	0.97760	rs61875309
10:100003242	t	g	-0.0305	0.0546	0.5766	-+	0.0	0.199	1	0.6555	0.10543	0.55051	rs12258651
10:100003785	t	c	0.0383	0.0258	0.1374	+-	65.5	2.899	1	0.08862	0.37275	0.80871	rs1359508
10:100004360	a	g	0.0426	0.0308	0.1667	++	0.0	0.504	1	0.4777	0.19402	0.97554	rs1048754
10:100004441	c	g	0.0373	0.0258	0.1482	+-	67.5	3.081	1	0.07921	0.62538	0.80952	rs1048757
10:100004906	a	c	0.0053	0.0254	0.8358	+-	80.5	5.135	1	0.02344	0.43014	0.82614	rs3750595
10:100004996	a	g	-0.0383	0.0258	0.1373	-+	65.5	2.898	1	0.08869	0.37271	0.80874	rs2025625
10:100005282	t	c	0.0063	0.0253	0.8019	+-	81.0	5.266	1	0.02175	0.43089	0.82667	rs10786405


In [5]:
%%bash
echo '
args <- commandArgs(trailingOnly = TRUE)
require("qqman");require("data.table");require("tidyr");require(dplyr)
FOLDER=args[1]
MODEL=args[2]
OUTCOME=args[3]
MH = fread(cmd = paste("cut -f 1,6,14 ",FOLDER, "/", MODEL, "/", OUTCOME, "/wID.txt", sep="")) %>%
  separate(MarkerName, c("CHR", "BP"), sep=":") %>% rename(P=`P-value`, SNP=ID) %>% 
  mutate_at(vars("CHR", "BP", "P"), as.numeric)
# lambda
chisq <- qchisq(1-median(MH$P),1)
lambda = median(chisq)/qchisq(0.5,1)
# QQ plot
png(paste(FOLDER, "/", MODEL, "/", OUTCOME, "/QQ.png", sep=""))
qq(MH$P, main = paste(OUTCOME, "in", MODEL))
text(4, 1, sprintf("lambda = %.3f", lambda))
text(4, 2, paste("N of variants =", nrow(MH)))
dev.off()
# MH plot if lambda is not too large.
if(lambda < 1.1){
    maxlogP=-log10(min(MH$P))
    png(paste(FOLDER, "/", MODEL, "/", OUTCOME,"/MH.png", sep=""), width=2000, height=1000, pointsize=18)
    manhattan(MH, col = c("blue4", "orange3"), main=paste(OUTCOME, "in", MODEL), cex.axis = 0.6, annotatePval = 0.00001, ylim=c(0, max(10, maxlogP)))
    dev.off()
}' > t/QQMH.R
Rscript --vanilla t/QQMH.R meta lnmxi logASYN 

null device 
          1 
null device 
          1 


Loading required package: qqman

For example usage please run: vignette('qqman')

Citation appreciated but not required:
Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).

Loading required package: data.table
Loading required package: tidyr
Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



QQ for Meta results (Case & Controls)
![QQplot](meta/lnmxi/logASYN/QQ.png)

MH for Case & Controls
![QQplot](meta/lnmxi/logASYN/MH.png)